In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, math, pandas, numpy.random as npr, multiprocessing as mp
from pylab import plot, show, legend
from time import time
from scipy.stats import *
from tqdm import trange
from ecology_functions import *

In [2]:
T = 50
I = 5  # number of locations
J = 3  # number of species
K = 2   # number of latent factors

In [3]:
lmbda = npr.randn(J,K)
alpha = npr.randn(J)
c = 0
phi = 0.5
logsigmasq = 0
x_0 = npr.randn(I,K)

theta = [alpha, lmbda, c, phi, logsigmasq]
Y, X = simulate_data(x_0, T, J, theta)

In [7]:
def f1(n_particles) :
    return block_pf(Y, x_0, n_particles, theta, calc_grad=True)[2]
def f2(n_particles) :
    return bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=True)[1]

n_particles = 1_000
rep = 1000

pool = mp.Pool(10)
results1 = pool.map(f1, [n_particles for n_particles in [n_particles]*rep])
results2 = pool.map(f2, [n_particles for n_particles in [n_particles]*rep])
pool.close()

alpha_grad = np.zeros((rep,2,J))
lmbda_grad = np.zeros((rep,2,J,K))
c_grad = np.zeros((rep,2))
phi_grad = np.zeros((rep,2))
logsigmasq_grad = np.zeros((rep,2))

for r in range(rep) :
    alpha_grad[r,0], lmbda_grad[r,0], c_grad[r,0], phi_grad[r,0], logsigmasq_grad[r,0] = results1[r][:]
    alpha_grad[r,1], lmbda_grad[r,1], c_grad[r,1], phi_grad[r,1], logsigmasq_grad[r,1] = results2[r][:]

In [6]:
for r in range(rep) :
    alpha_grad[r,0], lmbda_grad[r,0], c_grad[r,0], phi_grad[r,0], logsigmasq_grad[r,0] = results1[r][:]
    alpha_grad[r,1], lmbda_grad[r,1], c_grad[r,1], phi_grad[r,1], logsigmasq_grad[r,1] = results2[r][:]

ValueError: could not broadcast input array from shape (1,4) into shape (3,2)

In [7]:
results1[r]

[array([ 80.4917424 , 138.30149933, -54.15859258]),
 array([[5000.,   25.,   15.,   10.]]),
 -5.441453779864259,
 -490.8404482324528,
 249.28569085956673]

In [10]:
print(np.var(c_grad,0))
print(np.var(phi_grad,0))
print(np.var(logsigmasq_grad,0))

[  9.86946786 353.92798141]
[  96.22676876 1880.24696403]
[ 12.96996289 242.27504278]


In [11]:
print(np.mean(c_grad,0))
print(np.mean(phi_grad,0))
print(np.mean(logsigmasq_grad,0))

[7.65100359 4.56481109]
[-493.03929768 -449.70278331]
[250.52134758 237.02787687]


In [13]:
particles = npr.randn(n_particles,I,K)
propagated_particles = npr.randn(n_particles,I,K)

In [15]:
grad = np.zeros((n_particles,J,K,I))    

In [17]:
y = Y[0]
np.shape(y-1)

(5, 3)

In [19]:
n = 0
np.shape(propagated_particles[n])

(5, 2)

In [24]:
np.dot(y.transpose(), propagated_particles[n])

array([[ 4.08477223e-01, -9.80775853e-01],
       [ 2.28723624e-03, -1.38864589e-01],
       [-6.74407285e-01, -2.78141212e+00]])

In [29]:
res = np.zeros((J,K,I))
for j in range(J) :
    for k in range(K) :
        res[j,k] = y[:,j]*propagated_particles[n][:,k]

In [40]:
for j in range(J) :
    res[j,:] = (np.reshape(y[:,j],(I,1,n_particles))*(propagated_particles[n])).transpose()

In [62]:
resl = np.zeros((J,K,I,n_particles))
for j in range(J) :
    resl[j] = np.swapaxes(np.reshape(repmat(np.reshape(y[:,j]-1,(I,1)), 1, n_particles), (I,1,n_particles))\
              *np.swapaxes(propagated_particles.transpose(),0,1), 0,1)

In [69]:
np.shape(np.swapaxes(resl.transpose(),2,3))

(1000, 5, 3, 2)

In [71]:
weights = npr.rand(n_particles,I)

In [75]:
np.sum(np.swapaxes(resl.transpose(),2,3)*np.reshape(weights, (*np.shape(weights),1,1)),(0,1))

array([[-66.31418155,  -3.11051294],
       [-41.03654651, -19.88624812],
       [ -9.36372661,  15.15561567]])